In [1]:
import pandas as pd
from google.cloud import bigquery
import os
import numpy as np

In [2]:
# Initialize BigQuery client

client = bigquery.Client(project='physionet-data-416819')

In [3]:
# Creating a helper function to reuse the same code for different queries

def get_dataframe_using_query(query, client=client):
    """ Returns a dataframe by querying the database with the given query """
    return client.query(query).to_dataframe()

# Load a Dataframe of All Patients

In [4]:
# Get a dataframe of all Patients associated with an admission to the ICU.

all_patients_query = \
"""
SELECT * 
FROM `physionet-data.mimiciv_hosp.patients`
"""
df_all_patients = get_dataframe_using_query(all_patients_query)
df_all_patients.head()

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10078138,F,18,2110,2017 - 2019,NaT
1,10180372,M,18,2110,2008 - 2010,NaT
2,10686175,M,18,2110,2011 - 2013,NaT
3,10851602,F,18,2110,2014 - 2016,NaT
4,10902424,F,18,2110,2017 - 2019,NaT


# Load a Dataframe of All Patients Associated with a Hospital Stay

In [5]:
# Get a dataframe of all the hospital admissions associated with an ICU stay.

admissions_query = \
"""
SELECT * 
FROM `physionet-data.mimiciv_hosp.admissions`
"""

df_admissions = get_dataframe_using_query(admissions_query)
df_admissions.head(2)

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10106244,26713233,2147-05-09 10:34:00,2147-05-12 13:43:00,NaT,DIRECT EMER.,None,PHYSICIAN REFERRAL,HOME,Other,ENGLISH,SINGLE,WHITE,NaT,NaT,0
1,13700703,20448599,2172-09-25 01:01:00,2172-10-03 13:25:00,NaT,OBSERVATION ADMIT,None,EMERGENCY ROOM,HOME,Other,ENGLISH,MARRIED,WHITE,2172-09-24 17:38:00,2172-09-25 03:07:00,0


# Load a Dataframe of Patient Medical History

In [6]:
# Past medical history of the patients
    # Some information https://opendata.stackexchange.com/questions/9098/past-medical-medication-history-in-mimic-iii

past_medical_history_query = \
"""
SELECT ce.subject_id, ce.hadm_id, ce.charttime, ce.value 
FROM `physionet-data.mimiciv_icu.chartevents` ce 
INNER JOIN `physionet-data.mimiciv_icu.d_items` di 
ON ce.itemid = di.itemid 
AND di.itemid IN (225059, 225811);
"""

df_past_medical_history = get_dataframe_using_query(past_medical_history_query)
df_past_medical_history.head()

,subject_id,hadm_id,charttime,value
0,10283452,24697328,2122-09-08 01:27:00,MI
1,16892086,20574337,2183-06-16 22:40:00,MI
2,17955761,23940733,2184-04-15 05:35:00,MI
3,19469213,24832116,2153-01-27 04:36:00,MI
4,18536202,22001082,2175-01-20 18:44:00,MI


# Retrieve Hospital Admission Related Diagnoses

In [7]:
# Get all the diagnoses relating to a hospital admission coded using the ICD9 system.

icd_codes_with_description_query = \
"""
SELECT di.hadm_id, di.subject_id, di.icd_code, di.seq_num, dd.long_title
FROM `physionet-data.mimiciv_hosp.diagnoses_icd` di
INNER JOIN `physionet-data.mimiciv_hosp.d_icd_diagnoses` dd
ON di.icd_code = dd.icd_code
"""

df_icd_codes_with_description = get_dataframe_using_query(icd_codes_with_description_query)
df_icd_codes_with_description.head(3)

,hadm_id,subject_id,icd_code,seq_num,long_title
0,26381316,10000935,311,26,"Depressive disorder, not elsewhere classified"
1,26381316,10000935,78052,27,"Insomnia, unspecified"
2,25242409,10000980,58381,26,"Nephritis and nephropathy, not specified as ac..."


# Retrieve Lab Items

In [8]:
# Get the names of all the lab items

lab_items_query = \
"""
SELECT lab.itemid, lab.label, lab.fluid, lab.category
FROM `physionet-data.mimiciv_hosp.d_labitems` lab;
"""

df_lab_items = get_dataframe_using_query(lab_items_query)

## Remove spaces from the column names, so that we can use it for column names later
df_lab_items['label'] = df_lab_items['label'].str.replace(' ', '_')
df_lab_items['fluid'] = df_lab_items['fluid'].str.replace(' ', '_')

df_lab_items.head(3)


,itemid,label,fluid,category
0,52038,Base_Excess,Fluid,Blood Gas
1,52039,Calculated_Bicarbonate,Fluid,Blood Gas
2,52040,pCO2,Fluid,Blood Gas


# Load a Dataframe of all CKD Related Lab Events

In [10]:
# A list of lab items that can be of interest for CKD patients
itemid_lab_readings_of_interest = ['51464','51466','50808','50811','50853','50861','50862','50878','50882','50883','50884','50885','50889','50893','50902','50903','50904','50905','50906','50907','50912','50927','50930','50931','50970','50971','50976','50983','51000','51006','51007','51066','51067','51068','51069','51070','51073','51076','51077','51080','51081','51082','51095','51099','51102','51104','51105','51106','51133','51146','51199','51200','51222','51223','51224','51225','51244','51245','51246','51247','51248','51249','51250','51251','51252','51253','51254','51255','51256','51257','51265','51270','51271','51272','51273','51282','51283','51284','51285','51300','51301','51474','51492','51493','51494','51495']
list_lab_readings = list(map(int,itemid_lab_readings_of_interest))
    
# Split the item_ids list into chunks of specified size
chunk_size = 5
chunks = [list_lab_readings[i:i + chunk_size] for i in range(0, len(list_lab_readings), chunk_size)]

# Initialize empty list to hold DataFrames from each chunk
chunked_dataframes = []

for chunk in chunks:
    # Convert the current chunk into a continous string
    lab_query_list = ", ".join(map(str, chunk))
    print(lab_query_list) # print current chunk for progress tracking
    
    # Format SQL query for the current chunk
    lab_event_query = f"""
        SELECT le.subject_id, le.hadm_id, le.itemid, le.charttime, le.value, le.valueuom
        FROM `physionet-data.mimiciv_hosp.labevents` le 
        WHERE le.itemid IN ({lab_query_list});
    """
    
    # Execute query and append the result to the chunked_dataframes list
    df_chunk = get_dataframe_using_query(lab_event_query)
    chunked_dataframes.append(df_chunk)
    
# Concatenate all DataFrame chunks form the final DataFrame
df_lab_events = pd.concat(chunked_dataframes, ignore_index=True)
df_lab_events.head(3)

51464, 51466, 50808, 50811, 50853
50861, 50862, 50878, 50882, 50883
50884, 50885, 50889, 50893, 50902
50903, 50904, 50905, 50906, 50907
50912, 50927, 50930, 50931, 50970
50971, 50976, 50983, 51000, 51006
51007, 51066, 51067, 51068, 51069
51070, 51073, 51076, 51077, 51080
51081, 51082, 51095, 51099, 51102
51104, 51105, 51106, 51133, 51146
51199, 51200, 51222, 51223, 51224
51225, 51244, 51245, 51246, 51247
51248, 51249, 51250, 51251, 51252
51253, 51254, 51255, 51256, 51257
51265, 51270, 51271, 51272, 51273
51282, 51283, 51284, 51285, 51300
51301, 51474, 51492, 51493, 51494
51495


,subject_id,hadm_id,itemid,charttime,value,valueuom
0,10002428,23473524,50808,2156-05-12 00:11:00,1.18,mmol/L
1,10004235,<NA>,50808,2196-02-24 12:40:00,1.01,mmol/L
2,10005817,28661809,50808,2135-01-15 02:03:00,___,mmol/L


# Retrieve the CKD Lab Event Names and Item ID Codes 

In [11]:
# get the name of the CKD related lab events and their associated itemid codes
df_ckd_lab_items = df_lab_items[df_lab_items['itemid'].isin(list_lab_readings)].reset_index(drop=True)
df_ckd_lab_items.head(3)

,itemid,label,fluid,category
0,50808,Free_Calcium,Blood,Blood Gas
1,50811,Hemoglobin,Blood,Blood Gas
2,50853,25-OH_Vitamin_D,Blood,Chemistry


# Retrieve Height and Weight of All Patients

In [12]:
# query to extract all height and weight measurements for patients and combine them into a single dataframe
height_weight_query = """
WITH FirstVRawData AS (
    SELECT
        c.charttime,
        c.itemid,
        c.subject_id,
        c.hadm_id,
        CASE
            WHEN c.itemid IN (762, 763, 3723, 3580, 3581, 3582, 226512, 226531) THEN 'WEIGHT'
            WHEN c.itemid IN (920, 1394, 4187, 3486, 3485, 4188, 226707) THEN 'HEIGHT'
        END AS parameter,
        CASE
            WHEN c.itemid IN (3581, 226531) THEN c.valuenum * 0.45359237
            WHEN c.itemid IN (3582) THEN c.valuenum * 0.0283495231
            WHEN c.itemid IN (920, 1394, 4187, 3486, 226707) THEN c.valuenum * 2.54
            ELSE c.valuenum
        END AS valuenum
    FROM
        `physionet-data.mimiciv_icu.chartevents` c
    WHERE
        c.valuenum IS NOT NULL
        AND (
            (c.itemid IN (
                762, 763, 3723, 3580, 226512, -- Weight Kg
                3581, 226531,                -- Weight lb
                3582,                 -- Weight oz
                920, 1394, 4187, 3486, 226707, -- Height inches
                3485, 4188            -- Height cm
            ))
            AND c.valuenum <> 0
        )
),

-- Section 2: CTE - SingleParameters
SingleParameters AS (
    SELECT DISTINCT
        subject_id,
        hadm_id,
        parameter,
        first_value(valuenum) OVER (
            PARTITION BY subject_id, hadm_id, parameter
            ORDER BY charttime
            ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
        ) AS first_valuenum,
        MIN(valuenum) OVER (
            PARTITION BY subject_id, hadm_id, parameter
            ORDER BY charttime
            ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
        ) AS min_valuenum,
        AVG(valuenum) OVER (
            PARTITION BY subject_id, hadm_id, parameter
            ORDER BY charttime
            ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
        ) AS mean_valuenum,
        MAX(valuenum) OVER (
            PARTITION BY subject_id, hadm_id, parameter
            ORDER BY charttime
            ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
        ) AS max_valuenum
    FROM
        FirstVRawData
),

PivotParameters AS (
    SELECT
        subject_id,
        hadm_id,
        MAX(CASE WHEN parameter = 'HEIGHT' THEN first_valuenum ELSE NULL END) AS height_first,
        MAX(CASE WHEN parameter = 'HEIGHT' THEN min_valuenum ELSE NULL END) AS height_min,
        AVG(CASE WHEN parameter = 'HEIGHT' THEN mean_valuenum ELSE NULL END) AS height_mean,
        MAX(CASE WHEN parameter = 'HEIGHT' THEN max_valuenum ELSE NULL END) AS height_max,
        MAX(CASE WHEN parameter = 'WEIGHT' THEN first_valuenum ELSE NULL END) AS weight_first,
        MAX(CASE WHEN parameter = 'WEIGHT' THEN min_valuenum ELSE NULL END) AS weight_min,
        AVG(CASE WHEN parameter = 'WEIGHT' THEN mean_valuenum ELSE NULL END) AS weight_mean,
        MAX(CASE WHEN parameter = 'WEIGHT' THEN max_valuenum ELSE NULL END) AS weight_max
    FROM
        SingleParameters
    GROUP BY
        subject_id,
        hadm_id
)

SELECT
    f.hadm_id,
    f.subject_id,
    ROUND(CAST(f.height_first AS NUMERIC), 2) AS height_first,
    ROUND(CAST(f.height_min AS NUMERIC), 2) AS height_min,
    ROUND(CAST(f.height_mean AS NUMERIC), 2) AS height_mean,
    ROUND(CAST(f.height_max AS NUMERIC), 2) AS height_max,
    ROUND(CAST(f.weight_first AS NUMERIC), 2) AS weight_first,
    ROUND(CAST(f.weight_min AS NUMERIC), 2) AS weight_min,
    ROUND(CAST(f.weight_mean AS NUMERIC), 2) AS weight_mean,
    ROUND(CAST(f.weight_max AS NUMERIC), 2) AS weight_max
FROM
    PivotParameters f
ORDER BY
    subject_id,
    hadm_id;
"""

In [13]:
# create a dataframe of height and weight measurements for patients (with min, max, mean, first values)
df_height_weight = get_dataframe_using_query(height_weight_query)
df_height_weight.head(3)

,hadm_id,subject_id,height_first,height_min,height_mean,height_max,weight_first,weight_min,weight_mean,weight_max
0,29079034,10000032,152.400000000,152.400000000,152.400000000,152.400000000,39.400000000,39.330000000,39.350000000,39.400000000
1,26913865,10000980,None,None,None,None,76.200000000,76.020000000,76.080000000,76.200000000
2,24597018,10001217,None,None,None,None,71.200000000,71.030000000,71.120000000,71.200000000


In [14]:
df_all_patients.to_pickle("df_all_patients.pkl")
df_admissions.to_pickle("df_admissions.pkl")
df_past_medical_history.to_pickle("df_past_medical_history.pkl")
df_icd_codes_with_description.to_pickle("df_icd_codes_with_description.pkl")
df_lab_events.to_pickle("df_lab_events.pkl")
df_lab_items.to_pickle("df_lab_items.pkl")
df_ckd_lab_items.to_pickle("df_ckd_lab_items.pkl")
df_height_weight.to_pickle("df_height_weight.pkl")